In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [3]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="New-Experiment")

'2'

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids = '1',
                  filter_string="metrics.rmse < 6.7",
                  run_view_type=ViewType.ACTIVE_ONLY,
                   max_results=5,
                   order_by=["metrics.rmse ASC"]
                  )

In [11]:
for run in runs:
    print(f"run id:{run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id:3f54708836814a1db99f96d7b5354090, rmse: 6.2782
run id:2ce6ec1f7ad2490b87858a85e4ff36de, rmse: 6.2822
run id:82e47fc437c0454ca887fc402eb195fb, rmse: 6.2913
run id:a7a089966f6c4cf2a987d34e78a2b8b3, rmse: 6.2930
run id:11a9fec3776d443eb16f2ec1046f1308, rmse: 6.2930


In [12]:
# To promote some of these to model registry
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = "82e47fc437c0454ca887fc402eb195fb"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name = "nyc-taxi-regressor")
# creating a new version of model

Successfully registered model 'nyc-taxi-regressor'.
2022/06/02 11:35:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1654184127354, current_stage='None', description=None, last_updated_timestamp=1654184127354, name='nyc-taxi-regressor', run_id='82e47fc437c0454ca887fc402eb195fb', run_link=None, source='./mlruns/1/82e47fc437c0454ca887fc402eb195fb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [14]:
# checking all the registered models
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654184127333, description=None, last_updated_timestamp=1654184127354, latest_versions=[<ModelVersion: creation_timestamp=1654184127354, current_stage='None', description=None, last_updated_timestamp=1654184127354, name='nyc-taxi-regressor', run_id='82e47fc437c0454ca887fc402eb195fb', run_link=None, source='./mlruns/1/82e47fc437c0454ca887fc402eb195fb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-regressor', tags={}>]

In [16]:
# get the versions of 'nyc-taxi-regressor' model
latest_versions = client.get_latest_versions(name="nyc-taxi-regressor")

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [21]:
model_name = "nyc-taxi-regressor"
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = 1,
    stage = "Staging",
    archive_existing_versions = False)

<ModelVersion: creation_timestamp=1654184127354, current_stage='Staging', description=None, last_updated_timestamp=1654186795273, name='nyc-taxi-regressor', run_id='82e47fc437c0454ca887fc402eb195fb', run_link=None, source='./mlruns/1/82e47fc437c0454ca887fc402eb195fb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [22]:
# get the versions of 'nyc-taxi-regressor' model
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging


In [23]:
from datetime import datetime as dt

In [25]:
client.update_model_version(
    name = model_name,
    version =1,
    description = f"The model version {model_version} was transitioned to {new_stage} on {dt.today().date()}")

<ModelVersion: creation_timestamp=1654184127354, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-06-02', last_updated_timestamp=1654186803480, name='nyc-taxi-regressor', run_id='82e47fc437c0454ca887fc402eb195fb', run_link=None, source='./mlruns/1/82e47fc437c0454ca887fc402eb195fb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>